The goal of this chapter is to compare the analytically derived $Q$ matrix (aka. Infinetestimal generator Matrix) of a CTMC model of a SIR disease process to data generated by a gillespie simulations of that model.

To begin, lets define the disease model.

The SIR compartmental model features 3 compartiments (Susceptible, Infectious, and Recovered) though which N individuals pass between (i.e. $S + I + R = N$) over time. To impliment this, lets define a type to represent the state of an SIR system at time t, $X(t) = \{S, I, R\}$.

In [ ]:
SIR_state = @NamedTuple begin
    t::Real
    S::Int
    I::Int
    R::Int
end

```{dot}
digraph SIR {
    rankdir = "LR"
    S->I
    I->R
}
```
Lets initialise a population of N = 100 individuals, one of who is infected and the remaining 99 are Susceptible along with a data frame to record the state at time t = 0.

In [ ]:
X0 = SIR_state([0.0,99,1,0])

using DataFrames

SIR_CTMC = DataFrame(
    t = Vector{Float64}(),
    S = Vector{Int64}(),
    I = Vector{Int64}(),
    R = Vector{Int64}())

push!(SIR_CTMC, X0)

 In an SIR system there are two possible events that can occur after a certain time $\Delta t$: infection (i.e. an individual moves from the Susceptible compartment to the Infectious compartment) $X(t + \Delta t) = \{s-1, i + 1, r\}$ and  recovery $X(t + \Delta t) = \{s, i-1, r +1\}$. Lets define these as triples which we can add to our SIR_State variable


In [ ]:
SIR_events = (
    infection = [-1, 1, 0],
    recovery = [0,-1,1]
)

Only one eveCTMC model of a SIR disease process.  which event will occur? Epidemiologcal theory tells us that, assuming homogenous mixing of infectious and susceptible individuals there will be $S \times I$ opportunities for contact. Only a proportion of these opporunities will result in infection, so let the parameter $\beta$, the 'transmission rate'  modulate this. 

In [ ]:
p = (
    β = 0.5,
    γ = 0.1
    )

Simiularly, we'll specify a constant parameter, $\lambda$, which will modulate the rate of recovery.
Lets define a 'transition' function which, given a state $X(t) = \{s, i,r\}$ will return the 'propensity' of each event occurring

In [ ]:
function Transition_Function(X, p)
    (β, γ) = p
    (t,S,I,R) = X
    N = S + I + R
    infection = (β/N)*S*I
    recovery = γ*I
    [infection,recovery]
end;

Ok thats enough setting up, lets simulate the evolution of the system throrugh time.

Delta t will have an exponential distribution with parameter $1/\sum{a+b}$

In [ ]:
using Distributions

Δt = rand(Exponential(1/sum(Transition_Function(X0,p))))

Next we need to determine which event will happen. The probability of a transition will be given by the proportion of the 

In [ ]:
Pt = Transition_Function(X0,p)
Pi = Pt[1]/sum(Pt)
Pr = Pt[2]/sum(Pt)

To sample one event with probability weighting according to its relative propensity,

In [ ]:
r= rand() 
i = 1
cw = Pt[1]
while cw < r && i < length(Pt)
    i += 1
    @inbounds cw += Pt[i]
end

Δx = SIR_events[i]

Xt = SIR_state(collect(X0) + [Δt; Δx])

push!(SIR_CTMC, Xt)

We're going to want to run this stepping process a number of times, so lets repeat the process until some specified time ($t_{end}$) is reached

In [ ]:
tend = 50
t = Xt[1]

while t < tend
    Δt = rand(Exponential(1/sum(Transition_Function(Xt, p))))

    Pt = Transition_Function(Xt,p)
    Ptn = Pt ./ sum(Pt)

    i = 1
    cw = Ptn[1]
    while cw < rand() && i < length(Ptn)
        i += 1
        @inbounds cw += Ptn[i]
    end

    Δx = SIR_events[i]

    Xt = SIR_state(collect(Xt) + [Δt; Δx])

    push!(SIR_CTMC, Xt)

    t = Xt[1]
end

lets plot the results for a clearer picture

In [ ]:
using StatsPlots

@df SIR_CTMC plot(:t, [:S, :I, :R] )

The probability that a susceptible individual will become infected $p_{s-1, i+1}\leftarrow{s, i} = \frac$



Since $R = S+I -N$, se can represent the state of the system with $\{S, I\}$, where $S \in \{0,1, ... , N-I-R\}$, $I \in \{0,1, ... , N-S-R\}$.

for example, when N = 3, we can represent the system with a graph:

As a discrete time process this Markov chain can be represented by the transition matrix 
$$
p = \begin{bmatrix}
\end{bmatrix}
$$


looks reasonable. How about in comparison to the deterministic ODE solution


In [ ]:
using DifferentialEquations

function SIRModel!(du, u, p, t)
    S, I, R = u
    β, γ = p
    du[1] = -β*S*I
    du[2] = β*S*I-γ*I
    du[3] = γ*I
end 

p = [.005,.1]
u0 = [99,1,0]
tspan = (0.0, 50)

prob = ODEProblem(SIRModel!, u0, tspan, p)

sol = solve(prob)

plot!(sol)

They're not identical but maybe this is to be expected given the small sample size. We could inrease the sample size (which extended to \infty would equal the ODE), but lets instead perform the N = 100 simulation a few more times.

For convenience I'll wrap the whole simulation in a function which takes initial conditions and parameters in and outputs the data

In [ ]:
function SIR_sim(X0::Vector{Int64}, tmax, p)
    N = sum(X0)
    Xt = SIR_state([0; X0])

    SIR_events = (
        infection = [-1, 1, 0],
        recovery = [0,-1,1]
    )

    function Transition_Function(X, p)
        (β, γ) = p
        (t, S,I,R) = X
        infection = β/N*S*I
        recovery = γ*I
        [infection,recovery]
    end

    SIR_CTMC = DataFrame(
    t = Vector{Float64}(),
    S = Vector{Int64}(),
    I = Vector{Int64}(),
    R = Vector{Int64}())


    push!(SIR_CTMC, Xt)
    t = Xt[1]
    tmax = tmax
    while t < tmax
        Pt = Transition_Function(Xt, p)
        Ptn = Pt ./ sum(Pt)

        i = 1
        cw = Ptn[1]
        while cw < rand() && i < length(Ptn)
            i += 1
            @inbounds cw += Ptn[i]
        end

        Δx = SIR_events[i]
        Δt = rand(Exponential(1/sum(Pt)))

        Xt = SIR_state(collect(Xt) + [Δt; Δx])

        push!(SIR_CTMC, Xt)

        t += Δt
   

    end
     return SIR_CTMC
end

and run this 100 times

In [ ]:
tmax = 50.0
X0 = [99,1,0]
params = [.5, .1]
Nsims = 100
Sims = DataFrame(
    t = Vector{Vector{Float64}}(),
    S = Vector{Vector{Int64}}(),
    I = Vector{Vector{Int64}}(),
    R = Vector{Vector{Int64}}()) 

for i in 1:Nsims
    sim = SIR_sim(X0, tmax, params)
    push!(Sims.t, sim.t)
    push!(Sims.S, sim.S)
    push!(Sims.I, sim.I)
    push!(Sims.R, sim.R)
end

I'll also remove any samples in which the epidemic does not spread (i.e. when i = 0 and s != 0)

In [ ]:
extinction_i = falses(Nsims)
for i in eachindex(Sims.t)
    extinction_i[i] = false
    extinction = false
    while extinction != true 
        for j in eachindex(Sims.S[i])
            if Sims.I[i][j] == 0 && Sims.S[i][j] != 0
                extinction_i[i] = true
                extinction = true
            elseif  j == lastindex(Sims.t[i])
                extinction = true
                break
            end
        end
    end
end


Sims = Sims[extinction_i .== false, :] 

plot()
for i in 1:length(Sims.t)
   global Ploto = @df Sims plot!(:t[i], [:S[i] :I[i] :R[i]], 
    vars = [:t, :S, :I, :R], linecolor = [:blue :red :green], linealpha = 0.1, legend = false, xlim = [0,tmax]) 
end

Ploto

To summarise our data, it will be useful to be able to produce the distribution of states (across a number of samples) at a particular time t. I'll write a function to do this

In [ ]:
stateatt = function(t, Sims)

    function lastindexbeoforet(t, V)
        i = firstindex(V)
        while V[i] <  t
            i +=1
        end
        i-1
    end

    tis = lastindexbeoforet.(fill(t, lastindex(Sims.t)), Sims.t)
[getindex.(Sims.S, tis),
 getindex.(Sims.I, tis),
 getindex.(Sims.R, tis)]
end

We can use this function to define an average over all the our samples throught the course of the simulation.

In [ ]:
t = 1:0.1:tmax

SIR_Mean = DataFrame( 
    t = Vector{Float64}(),
    S = Vector{Float64}(),
    I = Vector{Float64}(),
    R = Vector{Float64}())


for i in t
    Xt = stateatt(i, Sims)
    Xtbar = mean.(Xt)
    push!(SIR_Mean, [i;Xtbar]) 
end

In [ ]:
Ploto
@df SIR_Mean plot!(:t, [:S, :I, :R], vars = [:t, :S, :I, :R], linecolor = [:blue :red :green])

We can also check the distribution of states at any given time. lets check t = 15

In [ ]:
t = 10
S, I, R = stateatt(t, Sims)
Sims_plot_df = DataFrame(S = S, I = I, R = R) |> 
    x -> stack(x, 1:3)
@df Sims_plot_df density(:value, group = :variable)

A nice snapshot, but maybe we should animate the progression

In [ ]:
anim = @animate for i in 1:0.5:tmax
    S, I, R = stateatt(i, Sims)
    Sims_plot_df = DataFrame(S = S, I = I, R = R) |> 
        x -> stack(x, 1:3)
    @df Sims_plot_df density(:value, group = :variable, alpha = 0.5, xlims = [0, 100], bins = 50)
end

gif(anim)

In [ ]:
tmax = 50

anim = @animate for i in 1:0.5:tmax
    S, I, R = stateatt(i, Sims)
    Sims_plot_df = DataFrame(S = S, I = I, R = R) 
     @df Sims_plot_df histogram2d(:S, :I, xlims = [0, 100], ylims = [0,100], bins = (50,50), colour = :plasma)
end

gif(anim)